#  Use a local language model using a vector database with query documents for the language model.

## Import packages

In [ ]:
#LagChain components 
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

# Support for dataset retrieval with Hugging Face 
from datasets import load_dataset 

# Casio use the integracion Astra in LangChain 
import cassio
# PDF reader
from PyPDF2 import PdfReader

#environtment 
import os 

## Processing the documents

In [2]:
# Provide the path of pdf/file 
pdfreader = PdfReader("PyBoy.pdf")

In [3]:
from typing_extensions import Concatenate 
#read text from pdf
raw_text = ""
for i, page in enumerate(pdfreader.pages):
  content = page.extract_text()
  if content: 
    raw_text += content

## Configuring the database 

In [4]:
token = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
db_id = os.getenv("ASTRA_DB_ID")

cassio.init(token=token, database_id=db_id)

## Model and embedding

In [5]:
from langchain_community import embeddings
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings.ollama import OllamaEmbeddings

llm = ChatOllama(model="llama3",temperature=0.6,base_url="http://localhost:11434")
embedding = OllamaEmbeddings(model="nomic-embed-text") #This model has very good results when it comes to embedding, at the time of this test it had better results than OpenAI.

## Using the database and entering data into it

In [6]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name = "qa_mini_demo",
    session=None,
    keyspace=None, 
)

In [7]:
from langchain.text_splitter import CharacterTextSplitter
#We need to split the text using Character Text Split such that it should not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [8]:
astra_vector_store.add_texts(texts[:50])
print("Inserted %i headlines." % len(texts[:50]))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 50 headlines.


## Use the model and verify that it uses the data provided from the vector database.

In [9]:
firts_question = True
while True:
  if firts_question:
    query_text = input("\nEnter your question (or type 'quit' to exit)").strip()
  else:
    query_text = input("\nWhat's your next question (or type  'quit' to exit): ").strip()

  if query_text.lower() == "quit":
    break
  if query_text =="":
    continue
  first_question = False

  print("\nQuestion: \"%s\"" % query_text)
  answer = astra_vector_index.query(query_text, llm=llm).strip()
  print("ANSWER: \"%s\"\n" % answer)
  print("FIRTS DOCUMENTS BY RELEVANCE:")
  for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
    print("  [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))


Question: "What is emulation?"
ANSWER: "Emulation is a process where one system or device mimics the behavior of another system or device. In other words, it's a software or hardware that replicates the functionality of another system, allowing it to run on a different platform.

In the context of retro gaming, for example, an emulator would allow you to play classic video games from older consoles or computers on your modern PC or phone, without having to own the original hardware. The emulator would mimic the behavior of the original console or computer, so that the game thinks it's running on the original hardware.

Emulation can be used in many areas:

1. **Retro gaming**: Emulate classic video games and play them on modern devices.
2. **Hardware preservation**: Preserve old technology by emulating its functionality, allowing us to study and learn from it without having to maintain the original hardware.
3. **Software testing**: Use emulation to test software compatibility with di

KeyboardInterrupt: Interrupted by user